In [1]:
import sys

sys.path.append("../../scripts")
import os
import pandas as pd
from pathlib import Path

from half_life_estimation import calc_gradient_zero_cell

In [2]:
def compile_table_publ(
    infile_halflife,
    infile_nad_amounts,
    mapping_halflife=None,
    mapping_nad_amounts=None,
    presicion=None,
    cyto_mito=False,
    outfile=None,
):
    """Compile table with half-life times, factors and turnover rate
    
    Loads calculated factors and half-life times and calculates turnover rates
    to compose tabel for publication
    :param infile_halflife: str or Path
        Filepath to half-life tables
    :param infile_nad_amounts: str or Path
        Filepath to NAD concentration/amount table
    :param mapping_halflife: dict
        Mapping to table index and half_life index
    :param mapping_nad_amounts: dict
        Mapping to table index and NAD concentration index
    :param presicion: dict
        Keys: column headers of return table
        Values: Number of digits to display
    :param cyto_mito: boolean (default: False)
        Show separate cyto, mito/cell values
    :param outfile: str or Path
        Filepath to save csv output to
    :return: pandas DataFrame
        Compiled DataFrame
    """
    df_hl = pd.read_csv(infile_halflife, index_col=0)
    if mapping_halflife:
        df_hl.rename(
            index=mapping_halflife, inplace=True,
        )

    # total intracellular NAD concentrations and SDs in nmol/mg protein
    nad_meas = pd.read_excel(
        Path(infile_nad_amounts[0]),
        sheet_name=infile_nad_amounts[1],
        nrows=10,
        skiprows=3,
        usecols=[1, 2, 3],
        index_col=0,
    )
    if mapping_nad_amounts:
        nad_meas.rename(
            index=mapping_nad_amounts, inplace=True,
        )
    if cyto_mito:
        df_summary = df_hl
        df_summary.drop(
            [
                "std_cell_gradient_at_0h",
                "std_cyto_gradient_at_0h",
                "std_mito_gradient_at_0h",
            ],
            axis=1,
        )
        column_mapping = {
            "half_life_time_cyto": "Cytosolic Half Life",
            "half_life_time_mito": "Mitochondrial  Half Life",
            "exp_prefactor_cyto": "Cytosolic Turnover Rate k",
            "exp_prefactor_mito": "Mitochondrial Turnover Rate k",
            "std_exp_factor_cyto": "Cytosolic std(k)",
            "std_exp_factor_mito": "Mitochondrial std(k)",
            "prefactor_cyto": "Cytosolic Prefactor",
            "prefactor_mito": "Mitochondrial Prefactor",
            "std_prefactor_cyto": "Cytosolic std(prefactor)",
            "std_prefactor_mito": "Mitochondrial std(prefactor)",
            "cell_gradient_at_0h": "Cellular turnover at 0h [fmol/(h·cell)]",
            "cyto_gradient_at_0h": "Cytosolic turnover at 0h [fmol/(h·cell)]",
            "mito_gradient_at_0h": "Mitochondrial turnover at 0h [fmol/(h·cell)]",
            "std_cell_gradient_at_0h": "Cellular std(turnover)",
            "std_cyto_gradient_at_0h": "Cytosolic std(turnover)",
            "std_mito_gradient_at_0h": "Mitochondrial std(turnover)",
            "nad_amount_cyto": "Cytosolic NAD amount [fmol/cell]",
            "nad_amount_mito": "Mitochondrial NAD amount [fmol/cell]",
            "n_samples": "n",
        }
        df_summary.rename(
            columns=column_mapping, inplace=True,
        )
    else:  # not cyto_mito
        column_mapping = {
            "half_life_time": "Half Life",
            "standard_deviation": "std(t1/2)",
            "exp_prefactor": "Turnover Rate k",
            "std_exp_factor": "std(k)",
            "prefactor": "Prefactor",
            "std_prefactor": "std(prefactor)",
            "n_samples": "n",
            "mean (nmol/mg protein)": "NAD amount [nmol/mg protein]",
            "SD": "std(NAD)",
        }
        df_summary = pd.merge(
            df_hl, nad_meas, how="left", left_index=True, right_index=True
        )
        df_summary.rename(
            columns=column_mapping, inplace=True,
        )
        (
            df_summary["Turnover at 0h [nmol/(h·mg protein)]"],
            df_summary["std(turnover)"],
        ) = zip(
            *df_summary.apply(
                lambda row: calc_gradient_zero_cell(
                    row["Prefactor"],
                    row["std(prefactor)"],
                    row["Turnover Rate k"],
                    row["std(k)"],
                    row["NAD amount [nmol/mg protein]"],
                    row["std(NAD)"],
                ),
                axis=1,
            )
        )

    if presicion:
        df_summary = format_digits(df_summary, presicion)
    if outfile:
        print(f"Saving {outfile}")
        df_summary.to_csv(Path(outfile))
    return df_summary

In [3]:
def format_digits(df, presicion):
    df_copy = df.copy()
    for col, digits in presicion.items():
        df_copy[col] = df_copy[col].map(lambda x: f"{{0:.{digits}f}}".format(x))
    return df_copy

In [4]:
infile_hl = "../../half_life_times/labelling_experiments_cell_lines.csv"
infile_nad_amounts = ("../../data/NAD_conc_MS.xlsx", "Summary")
outfile = "table_cell_lines.csv"

mapping_halflife = {
    "293_?.?": "293",
    "CytoPARP_?.?": "cytoPARP",
    "ER_PARP_?.?": "erPARP",
    "mitoPARP_?.?": "mitoPARP",
    "pexPARP_?.?": "pexPARP",
}
mapping_nad_amounts = {
    293: "293",
    "mP": "mitoPARP",
    "cP": "cytoPARP",
    "pP": "pexPARP",
    "ERP": "erPARP",
}
presicion = {
    "Prefactor": 3,
    "std(prefactor)": 3,
    "Turnover Rate k": 3,
    "std(k)": 3,
    "n": 0,
    "NAD amount [nmol/mg protein]": 1,
    "std(NAD)": 1,
    "Turnover at 0h [nmol/(h·mg protein)]": 1,
    "std(turnover)": 1,
}

In [5]:
df = compile_table_publ(
    infile_hl,
    infile_nad_amounts,
    mapping_halflife=mapping_halflife,
    mapping_nad_amounts=mapping_nad_amounts,
    presicion=presicion,
    outfile=outfile,
)
df

Saving table_cell_lines.csv


,Half Life,std(t1/2),Prefactor,std(prefactor),Turnover Rate k,std(k),n,NAD amount [nmol/mg protein],std(NAD),Turnover at 0h [nmol/(h·mg protein)],std(turnover)
293,4h 22min,7min,1.032,0.009,0.166,0.004,6,18.0,2.8,3.1,0.2
cytoPARP,3h 25min,8min,0.996,0.013,0.202,0.007,6,10.9,1.3,2.2,0.1
erPARP,4h 11min,4min,0.994,0.005,0.164,0.002,7,11.2,0.6,1.8,0.1
mitoPARP,2h 13min,5min,0.995,0.015,0.311,0.011,6,10.2,1.6,3.2,0.2
pexPARP,2h 17min,5min,0.986,0.012,0.298,0.009,6,9.5,1.4,2.8,0.1


In [6]:
infile_hl = "../../half_life_times/labelling_experiments_cell_lines_wcl_mito_separation_split_approach.csv"
infile_nad_amounts = ("../../data/NAD_conc_MS.xlsx", "Summary")
outfile = "table_cell_lines_cyto_mito.csv"

mapping_nad_amounts = {
    293: "293",
    "mP": "mitoPARP",
    "pP": "pexPARP",
}
presicion = {
    "Cytosolic Turnover Rate k": 2,
    "Mitochondrial Turnover Rate k": 2,
    "Cytosolic Prefactor": 3,
    "Mitochondrial Prefactor": 3,
    "Cytosolic std(k)": 3,
    "Mitochondrial std(k)": 3,
    "Cytosolic std(prefactor)": 3,
    "Mitochondrial std(prefactor)": 3,
    "Cellular turnover at 0h [fmol/(h·cell)]": 2,
    "Cytosolic turnover at 0h [fmol/(h·cell)]": 2,
    "Mitochondrial turnover at 0h [fmol/(h·cell)]": 2,
    "Cellular std(turnover)": 2,
    "Cytosolic std(turnover)": 2,
    "Mitochondrial std(turnover)": 2,
    "Cytosolic NAD amount [fmol/cell]": 1,
    "Mitochondrial NAD amount [fmol/cell]": 1,
}

In [7]:
df = compile_table_publ(
    infile_hl,
    infile_nad_amounts,
    mapping_halflife=None,
    mapping_nad_amounts=mapping_nad_amounts,
    presicion=presicion,
    cyto_mito=True,
    outfile=outfile,
)
df

Saving table_cell_lines_cyto_mito.csv


,Cytosolic Half Life,Mitochondrial Half Life,Cytosolic Turnover Rate k,Mitochondrial Turnover Rate k,Cytosolic Prefactor,Mitochondrial Prefactor,Cytosolic std(k),Mitochondrial std(k),Cytosolic std(prefactor),Mitochondrial std(prefactor),Cellular turnover at 0h [fmol/(h·cell)],Cytosolic turnover at 0h [fmol/(h·cell)],Mitochondrial turnover at 0h [fmol/(h·cell)],Cellular std(turnover),Cytosolic std(turnover),Mitochondrial std(turnover),Cytosolic NAD amount [fmol/cell],Mitochondrial NAD amount [fmol/cell]
293,4h 38min,5h 33min,0.15,0.13,1.002,1.027,0.004,0.004,0.009,0.012,0.71,0.50,0.21,0.70,0.30,0.63,3.3,1.6
pexPARP,2h 48min,3h 18min,0.25,0.22,1.002,1.019,0.005,0.006,0.008,0.013,0.63,0.53,0.10,2.27,0.47,2.22,2.1,0.5
mitoPARP,2h 52min,3h 43min,0.24,0.19,1.006,1.032,0.005,0.008,0.010,0.017,0.67,0.60,0.06,3.20,0.41,3.17,2.5,0.3
